**Importing libraries**


In [55]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import nltk

**Importing data from destination folder** here, Google Drive

In [56]:
df= pd.read_excel("/content/drive/MyDrive/Black_coffer/cik_list.xlsx")

 ..................................................................................................

In [57]:
#addding the link address as a prefix to the values in the SECFNAME column
'''
#example
a=df.SECFNAME[0]
str= "https://www.sec.gov/Archives/"+a
 
'''
df["SECFNAME"] =  "https://www.sec.gov/Archives/" + df['SECFNAME']

In [58]:
df.SECFNAME[0]

'https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-000413.txt'

In [60]:
text_array=[]
document_length=[]
for i in df.SECFNAME:
  response = requests.get(i)
  #scraping the pages one by one
  soup = BeautifulSoup(response.text, "html.parser")
  text = str(soup).lower()
  doc_length = len(text)
  text_array.append(text)
  document_length.append(doc_length)

In [61]:
len(document_length)

152


here it is apparent that the urls have a problem with fetching the information and hence a error message is returned as response which is evident from the fact that the size of many responses are same i,e, 4788

Functions defined to ease up the opeartions and calculation of variables

In [62]:
def mdaa(text, doc_length):
  first_instance_part1 = text.find("management's discussion and analysis")+1
  if first_instance_part1 != 0:
    new_text = text[(first_instance_part1)*3:doc_length]#skipping the index
    second_instance_part1 = new_text.find("management's discussion and analysis")#title reference
    text_forFlag = text[(first_instance_part1)*3 +second_instance_part1:doc_length]
    flag = text_forFlag.find("item ")
    try:
      mda1 = (text_forFlag[flag+5])# to check if its a title not a word
      mda = (text_forFlag[:flag])
      return mda
    except IndexError:
      return 0
  else:
    return 0
 
def qqdmr(text, doc_length):
  first_instance_part2 = text.find("quantitative and qualitative disclosures")+1
  if first_instance_part2 != 0:
    new_text = text[(first_instance_part2)*3:doc_length]
    second_instance_part2 = new_text.find("management's discussion and analysis")
    text_forFlag = text[(first_instance_part2)*3 +second_instance_part2:doc_length]
    flag = text_forFlag.find("item ")
    try:
      qqd1 = (text_forFlag[flag+5])
      qqd = (text_forFlag[:flag])
      return qqd
    except IndexError:
      return 0
  else:
    return 0
 
def rf(text, doc_length):
  first_instance_part3 = text.find("risk factors")+1
  if first_instance_part3 != 0:
    new_text = text[(first_instance_part3)*3:doc_length]
    second_instance_part3 = new_text.find("management's discussion and analysis")
    text_forFlag = text[(first_instance_part3)*3 +second_instance_part3:doc_length]
    flag = text_forFlag.find("item ")
    rfs = (text_forFlag[:flag])
    return rfs
  else:
    return 0
 
from nltk.corpus import stopwords
nltk.download("stopwords")
def clean_text(Ctext):
  Ctext = Ctext.lower()
  from bs4 import BeautifulSoup
  Ctext = BeautifulSoup(Ctext, features="lxml").get_text()
  from nltk.tokenize import RegexpTokenizer
  tokenizer = RegexpTokenizer(r'\w+')
  Ctext = tokenizer.tokenize(Ctext)
  clean = [word for word in Ctext if word not in stopwords.words('english')]
  return clean
 
def positive_words():
  myfile = open('/content/drive/MyDrive/Black_coffer/PositiveWords.txt', "r")
  positives = myfile.readlines()
  positive = [pos.strip().lower() for pos in positives]
  return positive
 
def negative_words():
  myfile = open('/content/drive/MyDrive/Black_coffer/NegativeWords.txt', "r")
  negatives = myfile.readlines()
  negative = [neg.strip().lower() for neg in negatives]
  return negative
 
def constrain_words():
  myfile = open('/content/drive/MyDrive/Black_coffer/constraining_dictionary.txt', "r")
  constrains = myfile.readlines()
  constrain = [con.strip().lower() for con in constrains]
  return constrain
 
def uncertain_words():
  myfile = open('/content/drive/MyDrive/Black_coffer/uncertainty_dictionary.txt', "r")
  uncertains = myfile.readlines()
  uncertain = [un.strip().lower() for un in uncertains]
  return uncertain
 
def negative_word_counter(cleantext, negative):
  negs = [word for word in cleantext if word in negative]
  return len(negs)
 
def positive_word_counter(cleantext, positive):
  pos = [word for word in cleantext if word in positive]
  return len(pos)
 
def constrain_word_counter(cleantext, constrain):
  con = [word for word in cleantext if word in constrain]
  return len(con)

def uncertain_word_counter(cleantext, uncertain):
  unc = [word for word in cleantext if word in uncertain]
  return len(unc)

def positive_score(cleantext, positive):
  score=0
  for word in cleantext:
    if word in positive:
      score=score+1
  return score

def negative_score(cleantext, negative):
  score=0
  for word in cleantext:
    if word in negative:
      score=score-1
  return (score*(-1))
  
def getSentiment(cleantext, negative, positive):
  positive = positive_words()
  negative = negative_words()
  return ((positive_word_counter(cleantext, positive) - negative_word_counter(cleantext, negative))/(positive_word_counter(cleantext, positive) + negative_word_counter(cleantext, negative) + 0.000001))
 
def complex_word(words):
  ctr = 0
  for word in words:
    if len(word) > 2:
      ctr += 1
  return ctr

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
positive=positive_words()
negative= negative_words()
constrain= constrain_words()
uncertain=uncertain_words()
feature=[]

for i in range(0,len(df.SECFNAME)):
  mda=mdaa(text_array[i], document_length[i])
  qqd=qqdmr(text_array[i], document_length[i])
  rfs=rf(text_array[i], document_length[i])
  arr=[]
  if mda != 0 and qqd == 0 and rfs == 0:
    ct=clean_text(mda)
    mda_positive_score = positive_word_counter(ct, positive)#..............................................................positive score
    mda_negative_score = negative_word_counter(ct, negative)#..........................................................negative score
    mda_polarity= getSentiment(ct, negative, positive)#..........................................................................polarity score
    mda_subjectivity= (mda_positive_score+mda_negative_score)/ ((len(ct))+0.000001)
    mda_avg_sentence_length = len(mda.split())/mda.count('.')#...........................................................average sentence length
    mda_constrain_count = constrain_word_counter(ct, constrain)
    mda_uncertain_count = uncertain_word_counter(ct, uncertain)
    mda_word_count=len(ct)
    mda_complex_words = complex_word(ct)#.......................................................................................complex word count
    mda_word_count=len(ct)#..................................................................................................................word count
    mda_percentage_complex_words = ((mda_complex_words)/len(ct)*100)#.....................................percentage of complex word
    mda_fog_index=  0.4*(mda_avg_sentence_length + mda_percentage_complex_words)#...........fog index
    mda_uncertain_score= uncertain_word_counter(ct, uncertain)#.......................................................uncertainty score
    mda_constrain_score= constrain_word_counter(ct, constrain)#........................................................constraining score
    mda_positive_word_proportion= mda_positive_score/mda_word_count#.........................................positive word proportion
    mda_negative_word_proportion= mda_negative_score/mda_word_count#......................................negative word proportion
    mda_uncertainity_word_proportion= mda_uncertain_score/mda_word_count#................................uncertainity word proportion
    mda_constraining_word_proportion=mda_constrain_score/mda_word_count#.................................constraining word proportion

    qqd_positive_score =0
    qqd_negative_score = 0
    qqd_polarity= 0
    qqd_subjectivity=0
    qqd_avg_sentence_length = 0
    qqd_constrain_count = 0
    qqd_uncertain_count = 0
    qqd_word_count=0
    qqd_complex_words = 0
    qqd_word_count=0
    qqd_percentage_complex_words = 0
    qqd_fog_index=  0
    qqd_uncertain_score= 0
    qqd_constrain_score= 0
    qqd_positive_word_proportion= 0
    qqd_negative_word_proportion=0
    qqd_uncertainity_word_proportion= 0
    qqd_constraining_word_proportion=0

    rfs_positive_score =0
    rfs_negative_score = 0
    rfs_polarity= 0
    rfs_subjectivity=0
    rfs_avg_sentence_length = 0
    rfs_constrain_count = 0
    rfs_uncertain_count = 0
    rfs_word_count=0
    rfs_complex_words = 0
    rfs_word_count=0
    rfs_percentage_complex_words = 0
    rfs_fog_index=  0
    rfs_uncertain_score= 0
    rfs_constrain_score= 0
    rfs_positive_word_proportion= 0
    rfs_negative_word_proportion=0
    rfs_uncertainity_word_proportion= 0
    rfs_constraining_word_proportion=0

  if qqd!=0 and mda==0 and rfs==0:
    ct=clean_text(qqd)
    qqd_positive_score = positive_word_counter(ct, positive)#..............................................................positive score
    qqd_negative_score = negative_word_counter(ct, negative)#..........................................................negative score
    qqd_polarity= getSentiment(ct, negative, positive)#..........................................................................polarity score
    qqd_subjectivity= (qqd_positive_score+qqd_negative_score)/ ((len(ct))+0.000001)
    qqd_avg_sentence_length = len(qqd.split())/qqd.count('.')#...........................................................average sentence length
    qqd_constrain_count = constrain_word_counter(ct, constrain)
    qqd_uncertain_count = uncertain_word_counter(ct, uncertain)
    qqd_word_count=len(ct)
    qqd_complex_words = complex_word(ct)#.......................................................................................complex word count
    qqd_word_count=len(ct)#..................................................................................................................word count
    qqd_percentage_complex_words = (qqd_complex_words/len(ct)*100)#.....................................percentage of complex word
    qqd_fog_index=  0.4*(qqd_avg_sentence_length + qqd_percentage_complex_words)#...........fog index
    qqd_uncertain_score= uncertain_word_counter(ct, uncertain)#.......................................................uncertainty score
    qqd_constrain_score= constrain_word_counter(ct, constrain)#........................................................constraining score
    qqd_positive_word_proportion= qqd_positive_score/qqd_word_count#.........................................positive word proportion
    qqd_negative_word_proportion= qqd_negative_score/qqd_word_count#......................................negative word proportion
    qqd_uncertainity_word_proportion= qqd_uncertain_score/qqd_word_count#................................uncertainity word proportion
    qqd_constraining_word_proportion=qqd_constrain_score/qqd_word_count#................................constraining word proportion

    mda_positive_score =0
    mda_negative_score = 0
    mda_polarity= 0
    mda_subjectivity=0
    mda_avg_sentence_length = 0
    mda_constrain_count = 0
    mda_uncertain_count = 0
    mda_word_count=0
    mda_complex_words = 0
    mda_word_count=0
    mda_percentage_complex_words = 0
    mda_fog_index=  0
    mda_uncertain_score= 0
    mda_constrain_score= 0
    mda_positive_word_proportion= 0
    mda_negative_word_proportion=0
    mda_uncertainity_word_proportion= 0
    mda_constraining_word_proportion=0

    rfs_positive_score =0
    rfs_negative_score = 0
    rfs_polarity= 0
    rfs_subjectivity=0
    rfs_avg_sentence_length = 0
    rfs_constrain_count = 0
    rfs_uncertain_count = 0
    rfs_word_count=0
    rfs_complex_words = 0
    rfs_word_count=0
    rfs_percentage_complex_words = 0
    rfs_fog_index=  0
    rfs_uncertain_score= 0
    rfs_constrain_score= 0
    rfs_positive_word_proportion= 0
    rfs_negative_word_proportion=0
    rfs_uncertainity_word_proportion= 0
    rfs_constraining_word_proportion=0

  if mda==0 and qqd==0 and rfs!=0:
    mda_positive_score =0
    mda_negative_score = 0
    mda_polarity= 0
    mda_subjectivity=0
    mda_avg_sentence_length = 0
    mda_constrain_count = 0
    mda_uncertain_count = 0
    mda_word_count=0
    mda_complex_words = 0
    mda_word_count=0
    mda_percentage_complex_words = 0
    mda_fog_index=  0
    mda_uncertain_score= 0
    mda_constrain_score= 0
    mda_positive_word_proportion= 0
    mda_negative_word_proportion=0
    mda_uncertainity_word_proportion= 0
    mda_constraining_word_proportion=0

    ct=clean_text(rfs)
    rfs_positive_score = positive_word_counter(ct, positive)#..............................................................positive score
    rfs_negative_score = negative_word_counter(ct, negative)#..........................................................negative score
    rfs_polarity= getSentiment(ct, negative, positive)#..........................................................................polarity score
    rfs_subjectivity= (rfs_positive_score+rfs_negative_score)/ ((len(ct))+0.000001)
    rfs_avg_sentence_length = len(rfs.split())/rfs.count('.')#...........................................................average sentence length
    rfs_constrain_count = constrain_word_counter(ct, constrain)
    rfs_uncertain_count = uncertain_word_counter(ct, uncertain)
    rfs_word_count=len(ct)
    rfs_complex_words = complex_word(ct)#.......................................................................................complex word count
    rfs_word_count=len(ct)#..................................................................................................................word count
    rfs_percentage_complex_words = ((rfs_complex_words)/len(ct)*100)#.....................................percentage of complex word
    rfs_fog_index=  0.4*(rfs_avg_sentence_length + rfs_percentage_complex_words)#...........fog index
    rfs_uncertain_score= uncertain_word_counter(ct, uncertain)#.......................................................uncertainty score
    rfs_constrain_score= constrain_word_counter(ct, constrain)#........................................................constraining score
    rfs_positive_word_proportion= rfs_positive_score/rfs_word_count#.........................................positive word proportion
    rfs_negative_word_proportion= rfs_negative_score/rfs_word_count#......................................negative word proportion
    rfs_uncertainity_word_proportion= rfs_uncertain_score/rfs_word_count#................................uncertainity word proportion
    rfs_constraining_word_proportion=rfs_constrain_score/rfs_word_count#................................constraining word proportion
    
    mda_positive_score =0
    mda_negative_score = 0
    mda_polarity= 0
    mda_subjectivity=0
    mda_avg_sentence_length = 0
    mda_constrain_count = 0
    mda_uncertain_count = 0
    mda_word_count=0
    mda_complex_words = 0
    mda_word_count=0
    mda_percentage_complex_words = 0
    mda_fog_index=  0
    mda_uncertain_score= 0
    mda_constrain_score= 0
    mda_positive_word_proportion= 0
    mda_negative_word_proportion=0
    mda_uncertainity_word_proportion= 0
    mda_constraining_word_proportion=0

    qqd_positive_score =0
    qqd_negative_score = 0
    qqd_polarity= 0
    qqd_subjectivity=0
    qqd_avg_sentence_length = 0
    qqd_constrain_count = 0
    qqd_uncertain_count = 0
    qqd_word_count=0
    qqd_complex_words = 0
    qqd_word_count=0
    qqd_percentage_complex_words = 0
    qqd_fog_index=  0
    qqd_uncertain_score= 0
    qqd_constrain_score= 0
    qqd_positive_word_proportion= 0
    qqd_negative_word_proportion=0
    qqd_uncertainity_word_proportion= 0
    qqd_constraining_word_proportion=0

  if mda==0 and qqd!=0 and rfs!=0:
    mda_positive_score =0
    mda_negative_score = 0
    mda_polarity= 0
    mda_subjectivity=0
    mda_avg_sentence_length = 0
    mda_constrain_count = 0
    mda_uncertain_count = 0
    mda_word_count=0
    mda_complex_words = 0
    mda_word_count=0
    mda_percentage_complex_words = 0
    mda_fog_index=  0
    mda_uncertain_score= 0
    mda_constrain_score= 0
    mda_positive_word_proportion= 0
    mda_negative_word_proportion=0
    mda_uncertainity_word_proportion= 0
    mda_constraining_word_proportion=0

    ct=clean_text(rfs)
    rfs_positive_score = positive_word_counter(ct, positive)#..............................................................positive score
    rfs_negative_score = negative_word_counter(ct, negative)#..........................................................negative score
    rfs_polarity= getSentiment(ct, negative, positive)#..........................................................................polarity score
    rfs_subjectivity= (rfs_positive_score+rfs_negative_score)/ ((len(ct))+0.000001)
    rfs_avg_sentence_length = len(rfs.split())/rfs.count('.')#...........................................................average sentence length
    rfs_constrain_count = constrain_word_counter(ct, constrain)
    rfs_uncertain_count = uncertain_word_counter(ct, uncertain)
    rfs_word_count=len(ct)
    rfs_complex_words = complex_word(ct)#.......................................................................................complex word count
    rfs_word_count=len(ct)#..................................................................................................................word count
    rfs_percentage_complex_words = ((rfs_complex_words)/len(ct)*100)#.....................................percentage of complex word
    rfs_fog_index=  0.4*(rfs_avg_sentence_length + rfs_percentage_complex_words)#...........fog index
    rfs_uncertain_score= uncertain_word_counter(ct, uncertain)#.......................................................uncertainty score
    rfs_constrain_score= constrain_word_counter(ct, constrain)#........................................................constraining score
    rfs_positive_word_proportion= rfs_positive_score/rfs_word_count#.........................................positive word proportion
    rfs_negative_word_proportion= rfs_negative_score/rfs_word_count#......................................negative word proportion
    rfs_uncertainity_word_proportion= rfs_uncertain_score/rfs_word_count#................................uncertainity word proportion
    rfs_constraining_word_proportion=rfs_constrain_score/rfs_word_count#................................constraining word proportion

    ct=clean_text(qqd)
    qqd_positive_score = positive_word_counter(ct, positive)#..............................................................positive score
    qqd_negative_score = negative_word_counter(ct, negative)#..........................................................negative score
    qqd_polarity= getSentiment(ct, negative, positive)#..........................................................................polarity score
    qqd_subjectivity= (qqd_positive_score+qqd_negative_score)/ ((len(ct))+0.000001)
    qqd_avg_sentence_length = len(qqd.split())/qqd.count('.')#...........................................................average sentence length
    qqd_constrain_count = constrain_word_counter(ct, constrain)
    qqd_uncertain_count = uncertain_word_counter(ct, uncertain)
    qqd_word_count=len(ct)
    qqd_complex_words = complex_word(ct)#.......................................................................................complex word count
    qqd_word_count=len(ct)#..................................................................................................................word count
    qqd_percentage_complex_words = (qqd_complex_words/len(ct)*100)#.....................................percentage of complex word
    qqd_fog_index=  0.4*(qqd_avg_sentence_length + qqd_percentage_complex_words)#...........fog index
    qqd_uncertain_score= uncertain_word_counter(ct, uncertain)#.......................................................uncertainty score
    qqd_constrain_score= constrain_word_counter(ct, constrain)#........................................................constraining score
    qqd_positive_word_proportion= qqd_positive_score/qqd_word_count#.........................................positive word proportion
    qqd_negative_word_proportion= qqd_negative_score/qqd_word_count#......................................negative word proportion
    qqd_uncertainity_word_proportion= qqd_uncertain_score/qqd_word_count#................................uncertainity word proportion
    qqd_constraining_word_proportion=qqd_constrain_score/qqd_word_count#................................constraining word proportion

  if mda!=0 and qqd!=0 and rfs==0:
    ct=clean_text(qqd)
    qqd_positive_score = positive_word_counter(ct, positive)#..............................................................positive score
    qqd_negative_score = negative_word_counter(ct, negative)#..........................................................negative score
    qqd_polarity= getSentiment(ct, negative, positive)#..........................................................................polarity score
    qqd_subjectivity= (qqd_positive_score+qqd_negative_score)/ ((len(ct))+0.000001)
    qqd_avg_sentence_length = len(qqd.split())/qqd.count('.')#...........................................................average sentence length
    qqd_constrain_count = constrain_word_counter(ct, constrain)
    qqd_uncertain_count = uncertain_word_counter(ct, uncertain)
    qqd_word_count=len(ct)
    qqd_complex_words = complex_word(ct)#.......................................................................................complex word count
    qqd_word_count=len(ct)#..................................................................................................................word count
    qqd_percentage_complex_words = (qqd_complex_words/len(ct)*100)#.....................................percentage of complex word
    qqd_fog_index=  0.4*(qqd_avg_sentence_length + qqd_percentage_complex_words)#...........fog index
    qqd_uncertain_score= uncertain_word_counter(ct, uncertain)#.......................................................uncertainty score
    qqd_constrain_score= constrain_word_counter(ct, constrain)#........................................................constraining score
    qqd_positive_word_proportion= qqd_positive_score/qqd_word_count#.........................................positive word proportion
    qqd_negative_word_proportion= qqd_negative_score/qqd_word_count#......................................negative word proportion
    qqd_uncertainity_word_proportion= qqd_uncertain_score/qqd_word_count#................................uncertainity word proportion
    qqd_constraining_word_proportion=qqd_constrain_score/qqd_word_count#................................constraining word proportion

    ct=clean_text(mda)
    mda_positive_score = positive_word_counter(ct, positive)#..............................................................positive score
    mda_negative_score = negative_word_counter(ct, negative)#..........................................................negative score
    mda_polarity= getSentiment(ct, negative, positive)#..........................................................................polarity score
    mda_subjectivity= (mda_positive_score+mda_negative_score)/ ((len(ct))+0.000001)
    mda_avg_sentence_length = len(mda.split())/mda.count('.')#...........................................................average sentence length
    mda_constrain_count = constrain_word_counter(ct, constrain)
    mda_uncertain_count = uncertain_word_counter(ct, uncertain)
    mda_word_count=len(ct)
    mda_complex_words = complex_word(ct)#.......................................................................................complex word count
    mda_word_count=len(ct)#..................................................................................................................word count
    mda_percentage_complex_words = ((mda_complex_words)/len(ct)*100)#.....................................percentage of complex word
    mda_fog_index=  0.4*(mda_avg_sentence_length + mda_percentage_complex_words)#...........fog index
    mda_uncertain_score= uncertain_word_counter(ct, uncertain)#.......................................................uncertainty score
    mda_constrain_score= constrain_word_counter(ct, constrain)#........................................................constraining score
    mda_positive_word_proportion= mda_positive_score/mda_word_count#.........................................positive word proportion
    mda_negative_word_proportion= mda_negative_score/mda_word_count#......................................negative word proportion
    mda_uncertainity_word_proportion= mda_uncertain_score/mda_word_count#................................uncertainity word proportion
    mda_constraining_word_proportion=mda_constrain_score/mda_word_count#.................................constraining word proportion

    rfs_positive_score =0
    rfs_negative_score = 0
    rfs_polarity= 0
    rfs_subjectivity=0
    rfs_avg_sentence_length = 0
    rfs_constrain_count = 0
    rfs_uncertain_count = 0
    rfs_word_count=0
    rfs_complex_words = 0
    rfs_word_count=0
    rfs_percentage_complex_words = 0
    rfs_fog_index=  0
    rfs_uncertain_score= 0
    rfs_constrain_score= 0
    rfs_positive_word_proportion= 0
    rfs_negative_word_proportion=0
    rfs_uncertainity_word_proportion= 0
    rfs_constraining_word_proportion=0

  if mda!=0 and qqd==0 and rfs!=0:
    qqd_positive_score =0
    qqd_negative_score = 0
    qqd_polarity= 0
    qqd_subjectivity=0
    qqd_avg_sentence_length = 0
    qqd_constrain_count = 0
    qqd_uncertain_count = 0
    qqd_word_count=0
    qqd_complex_words = 0
    qqd_word_count=0
    qqd_percentage_complex_words = 0
    qqd_fog_index=  0
    qqd_uncertain_score= 0
    qqd_constrain_score= 0
    qqd_positive_word_proportion= 0
    qqd_negative_word_proportion=0
    qqd_uncertainity_word_proportion= 0
    qqd_constraining_word_proportion=0

    ct=clean_text(mda)
    mda_positive_score = positive_word_counter(ct, positive)#..............................................................positive score
    mda_negative_score = negative_word_counter(ct, negative)#..........................................................negative score
    mda_polarity= getSentiment(ct, negative, positive)#..........................................................................polarity score
    mda_subjectivity= (mda_positive_score+mda_negative_score)/ ((len(ct))+0.000001)
    mda_avg_sentence_length = len(mda.split())/mda.count('.')#...........................................................average sentence length
    mda_constrain_count = constrain_word_counter(ct, constrain)
    mda_uncertain_count = uncertain_word_counter(ct, uncertain)
    mda_word_count=len(ct)
    mda_complex_words = complex_word(ct)#.......................................................................................complex word count
    mda_word_count=len(ct)#..................................................................................................................word count
    mda_percentage_complex_words = ((mda_complex_words)/len(ct)*100)#.....................................percentage of complex word
    mda_fog_index=  0.4*(mda_avg_sentence_length + mda_percentage_complex_words)#...........fog index
    mda_uncertain_score= uncertain_word_counter(ct, uncertain)#.......................................................uncertainty score
    mda_constrain_score= constrain_word_counter(ct, constrain)#........................................................constraining score
    mda_positive_word_proportion= mda_positive_score/mda_word_count#.........................................positive word proportion
    mda_negative_word_proportion= mda_negative_score/mda_word_count#......................................negative word proportion
    mda_uncertainity_word_proportion= mda_uncertain_score/mda_word_count#................................uncertainity word proportion
    mda_constraining_word_proportion=mda_constrain_score/mda_word_count#.................................constraining word proportion

    ct=clean_text(rfs)
    rfs_positive_score = positive_word_counter(ct, positive)#..............................................................positive score
    rfs_negative_score = negative_word_counter(ct, negative)#..........................................................negative score
    rfs_polarity= getSentiment(ct, negative, positive)#..........................................................................polarity score
    rfs_subjectivity= (rfs_positive_score+rfs_negative_score)/ ((len(ct))+0.000001)
    rfs_avg_sentence_length = len(rfs.split())/rfs.count('.')#...........................................................average sentence length
    rfs_constrain_count = constrain_word_counter(ct, constrain)
    rfs_uncertain_count = uncertain_word_counter(ct, uncertain)
    rfs_word_count=len(ct)
    rfs_complex_words = complex_word(ct)#.......................................................................................complex word count
    rfs_word_count=len(ct)#..................................................................................................................word count
    rfs_percentage_complex_words = ((rfs_complex_words)/len(ct)*100)#.....................................percentage of complex word
    rfs_fog_index=  0.4*(rfs_avg_sentence_length + rfs_percentage_complex_words)#...........fog index
    rfs_uncertain_score= uncertain_word_counter(ct, uncertain)#.......................................................uncertainty score
    rfs_constrain_score= constrain_word_counter(ct, constrain)#........................................................constraining score
    rfs_positive_word_proportion= rfs_positive_score/rfs_word_count#.........................................positive word proportion
    rfs_negative_word_proportion= rfs_negative_score/rfs_word_count#......................................negative word proportion
    rfs_uncertainity_word_proportion= rfs_uncertain_score/rfs_word_count#................................uncertainity word proportion
    rfs_constraining_word_proportion=rfs_constrain_score/rfs_word_count#................................constraining word proportion

  if mda!=0 and qqd!=0 and rfs!=0:
    ct=clean_text(mda)
    mda_positive_score = positive_word_counter(ct, positive)#..............................................................positive score
    mda_negative_score = negative_word_counter(ct, negative)#..........................................................negative score
    mda_polarity= getSentiment(ct, negative, positive)#..........................................................................polarity score
    mda_subjectivity= (mda_positive_score+mda_negative_score)/ ((len(ct))+0.000001)
    mda_avg_sentence_length = len(mda.split())/mda.count('.')#...........................................................average sentence length
    mda_constrain_count = constrain_word_counter(ct, constrain)
    mda_uncertain_count = uncertain_word_counter(ct, uncertain)
    mda_word_count=len(ct)
    mda_complex_words = complex_word(ct)#.......................................................................................complex word count
    mda_word_count=len(ct)#..................................................................................................................word count
    mda_percentage_complex_words = ((mda_complex_words)/len(ct)*100)#.....................................percentage of complex word
    mda_fog_index=  0.4*(mda_avg_sentence_length + mda_percentage_complex_words)#...........fog index
    mda_uncertain_score= uncertain_word_counter(ct, uncertain)#.......................................................uncertainty score
    mda_constrain_score= constrain_word_counter(ct, constrain)#........................................................constraining score
    mda_positive_word_proportion= mda_positive_score/mda_word_count#.........................................positive word proportion
    mda_negative_word_proportion= mda_negative_score/mda_word_count#......................................negative word proportion
    mda_uncertainity_word_proportion= mda_uncertain_score/mda_word_count#................................uncertainity word proportion
    mda_constraining_word_proportion=mda_constrain_score/mda_word_count#.................................constraining word proportion

    ct=clean_text(rfs)
    rfs_positive_score = positive_word_counter(ct, positive)#..............................................................positive score
    rfs_negative_score = negative_word_counter(ct, negative)#..........................................................negative score
    rfs_polarity= getSentiment(ct, negative, positive)#..........................................................................polarity score
    rfs_subjectivity= (rfs_positive_score+rfs_negative_score)/ ((len(ct))+0.000001)
    rfs_avg_sentence_length = len(rfs.split())/rfs.count('.')#...........................................................average sentence length
    rfs_constrain_count = constrain_word_counter(ct, constrain)
    rfs_uncertain_count = uncertain_word_counter(ct, uncertain)
    rfs_word_count=len(ct)
    rfs_complex_words = complex_word(ct)#.......................................................................................complex word count
    rfs_word_count=len(ct)#..................................................................................................................word count
    rfs_percentage_complex_words = ((rfs_complex_words)/len(ct)*100)#.....................................percentage of complex word
    rfs_fog_index=  0.4*(rfs_avg_sentence_length + rfs_percentage_complex_words)#...........fog index
    rfs_uncertain_score= uncertain_word_counter(ct, uncertain)#.......................................................uncertainty score
    rfs_constrain_score= constrain_word_counter(ct, constrain)#........................................................constraining score
    rfs_positive_word_proportion= rfs_positive_score/rfs_word_count#.........................................positive word proportion
    rfs_negative_word_proportion= rfs_negative_score/rfs_word_count#......................................negative word proportion
    rfs_uncertainity_word_proportion= rfs_uncertain_score/rfs_word_count#................................uncertainity word proportion
    rfs_constraining_word_proportion=rfs_constrain_score/rfs_word_count#................................constraining word proportion

    ct=clean_text(qqd)
    qqd_positive_score = positive_word_counter(ct, positive)#..............................................................positive score
    qqd_negative_score = negative_word_counter(ct, negative)#..........................................................negative score
    qqd_polarity= getSentiment(ct, negative, positive)#..........................................................................polarity score
    qqd_subjectivity= (qqd_positive_score+qqd_negative_score)/ ((len(ct))+0.000001)
    qqd_avg_sentence_length = len(qqd.split())/qqd.count('.')#...........................................................average sentence length
    qqd_constrain_count = constrain_word_counter(ct, constrain)
    qqd_uncertain_count = uncertain_word_counter(ct, uncertain)
    qqd_word_count=len(ct)
    qqd_complex_words = complex_word(ct)#.......................................................................................complex word count
    qqd_word_count=len(ct)#..................................................................................................................word count
    qqd_percentage_complex_words = (qqd_complex_words/len(ct)*100)#.....................................percentage of complex word
    qqd_fog_index=  0.4*(qqd_avg_sentence_length + qqd_percentage_complex_words)#...........fog index
    qqd_uncertain_score= uncertain_word_counter(ct, uncertain)#.......................................................uncertainty score
    qqd_constrain_score= constrain_word_counter(ct, constrain)#........................................................constraining score
    qqd_positive_word_proportion= qqd_positive_score/qqd_word_count#.........................................positive word proportion
    qqd_negative_word_proportion= qqd_negative_score/qqd_word_count#......................................negative word proportion
    qqd_uncertainity_word_proportion= qqd_uncertain_score/qqd_word_count#................................uncertainity word proportion
    qqd_constraining_word_proportion=qqd_constrain_score/qqd_word_count#................................constraining word proportion
  if mda==0 and qqd==0 and rfs==0:
    mda_positive_score =0
    mda_negative_score = 0
    mda_polarity= 0
    mda_subjectivity=0
    mda_avg_sentence_length = 0
    mda_constrain_count = 0
    mda_uncertain_count = 0
    mda_word_count=0
    mda_complex_words = 0
    mda_word_count=0
    mda_percentage_complex_words = 0
    mda_fog_index=  0
    mda_uncertain_score= 0
    mda_constrain_score= 0
    mda_positive_word_proportion= 0
    mda_negative_word_proportion=0
    mda_uncertainity_word_proportion= 0
    mda_constraining_word_proportion=0

    qqd_positive_score =0
    qqd_negative_score = 0
    qqd_polarity= 0
    qqd_subjectivity=0
    qqd_avg_sentence_length = 0
    qqd_constrain_count = 0
    qqd_uncertain_count = 0
    qqd_word_count=0
    qqd_complex_words = 0
    qqd_word_count=0
    qqd_percentage_complex_words = 0
    qqd_fog_index=  0
    qqd_uncertain_score= 0
    qqd_constrain_score= 0
    qqd_positive_word_proportion= 0
    qqd_negative_word_proportion=0
    qqd_uncertainity_word_proportion= 0
    qqd_constraining_word_proportion=0

    rfs_positive_score =0
    rfs_negative_score = 0
    rfs_polarity= 0
    rfs_subjectivity=0
    rfs_avg_sentence_length = 0
    rfs_constrain_count = 0
    rfs_uncertain_count = 0
    rfs_word_count=0
    rfs_complex_words = 0
    rfs_word_count=0
    rfs_percentage_complex_words = 0
    rfs_fog_index=  0
    rfs_uncertain_score= 0
    rfs_constrain_score= 0
    rfs_positive_word_proportion= 0
    rfs_negative_word_proportion=0
    rfs_uncertainity_word_proportion= 0
    rfs_constraining_word_proportion=0



  arr.append(mda_positive_score)
  arr.append(mda_negative_score)
  arr.append(mda_polarity)
  arr.append(mda_avg_sentence_length)
  arr.append(mda_complex_words)
  arr.append(mda_word_count)
  arr.append(mda_percentage_complex_words)
  arr.append(mda_fog_index)
  arr.append(mda_uncertain_score)
  arr.append(mda_constrain_score)
  arr.append(mda_positive_word_proportion)
  arr.append(mda_negative_word_proportion)
  arr.append(mda_uncertainity_word_proportion)
  arr.append(mda_constraining_word_proportion)

  arr.append(qqd_positive_score)
  arr.append(qqd_negative_score)
  arr.append(qqd_polarity)
  arr.append(qqd_avg_sentence_length)
  arr.append(qqd_complex_words)
  arr.append(qqd_word_count)
  arr.append(qqd_percentage_complex_words)
  arr.append(qqd_fog_index)
  arr.append(qqd_uncertain_score)
  arr.append(qqd_constrain_score)
  arr.append(qqd_positive_word_proportion)
  arr.append(qqd_negative_word_proportion)
  arr.append(qqd_uncertainity_word_proportion)
  arr.append(qqd_constraining_word_proportion)

  arr.append(qqd_positive_score)
  arr.append(qqd_negative_score)
  arr.append(qqd_polarity)
  arr.append(qqd_avg_sentence_length)
  arr.append(qqd_complex_words)
  arr.append(qqd_word_count)
  arr.append(qqd_percentage_complex_words)
  arr.append(qqd_fog_index)
  arr.append(qqd_uncertain_score)
  arr.append(qqd_constrain_score)
  arr.append(qqd_positive_word_proportion)
  arr.append(qqd_negative_word_proportion)
  arr.append(qqd_uncertainity_word_proportion)
  arr.append(qqd_constraining_word_proportion)

  feature.append(arr)

  

Certain urls have no content and only html tags. Hence the length of the cleaned text is zero at some urls. hence certain operationss couldnot be proceeded with certain errors.
 Well by the code in the above segment, theoretically, the required values to the specified variables are added to the array, row wise.


Because of the above mentioned reason and certain problems with the text and the URL itself, I couldnt add the variables to the dataset.
